In [ ]:
using Random
using Statistics

using AbstractPlotting
using CairoMakie
using LightGraphs
using JuMP
using SCIP
using Triangle

In [ ]:
# create instance data
Random.seed!(0);

const N = 7
const WIDTH = 500
const HEIGHT = 300

x = 0.95 * WIDTH * rand(N)
y = 0.95 * HEIGHT * rand(N)

points = [x y]

In [ ]:
function make_scene(width=WIDTH, height=HEIGHT)
    return Scene(resolution=(width, height), show_axis=false, scale_plot=false)
end

In [ ]:
function draw_points!(points; markersize=4, color=:black)
    scatter!(points, markersize=markersize, color=color)
end

In [ ]:
make_scene()
draw_points!(points)

In [ ]:
struct Triangulation
    points::Matrix{Float64}   # n x 2
    edges::Matrix{Int64}      # m x 2
    triangles::Matrix{Int64}  # t x 3
end

In [ ]:
function unique_edges(triangles)
    set = Set()
    for t in 1:size(triangles, 1)
        triangle = triangles[t, :]
        push!(set, min(triangle[[1, 2]], triangle[[2, 1]]))
        push!(set, min(triangle[[2, 3]], triangle[[3, 2]]))
        push!(set, min(triangle[[1, 3]], triangle[[3, 1]]))
    end
    return hcat(sort(collect(set))...)'
end

In [ ]:
unique_edges(tt)

In [ ]:
function delaunay_triangulation(points)::Triangulation
    points_map = collect(1:size(points, 1))
    triangle_array = Triangle.basic_triangulation(points, points_map)
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(points, edges, triangles)
end

In [ ]:
function draw_edges!(triangulation; color=:gray)
    linesegments!(triangulation.points[triangulation.edges'[:], :], color=color)
end

In [ ]:
function draw_triangulation(triangulation)
    make_scene()
    draw_edges!(triangulation)
    draw_points!(triangulation.points)   
end

In [ ]:
del = delaunay_triangulation(points)
draw_triangulation(del)

In [ ]:
function triangle_centers(triangulation)
    points = triangulation.points
    triangles = triangulation.triangles
    return dropdims(mean(points[triangles, :], dims=2), dims=2)
end

In [ ]:
centers = triangle_centers(del)
draw_points!(centers, color=:limegreen)

In [ ]:
function delaunay_with_centers(triangulation)
    centers = triangle_centers(triangulation)
    all_points = vcat(triangulation.points, centers)
    return delaunay_triangulation(all_points)
end

In [ ]:
draw_triangulation(delaunay_with_centers(del))

In [ ]:
draw_triangulation(delaunay_with_centers(delaunay_with_centers(del)))

In [ ]:
# find edge centers for subdivision
edge_centers = mean(points[edges, :], dims=1)[1, :, :]

# triangulate with added center points and edge subdivision, keeping old edges
points4 = vcat(points, centers_array, edge_centers)
points_map4 = collect(1:size(points4, 1))
triangles4 = constrained_triangulation(points4, points_map4, edges'[:,:])
edges4 = unique_edges(triangles4)

make_scene()
linesegments!(points4[vec(edges4), :], color=:orange)
linesegments!(points[vec(edges), :])
scatter!(edge_centers, markersize=4, color=:green)
scatter!(centers_array, markersize=4, color=:cyan)
scatter!(x, y, markersize=5, color=:red)

In [ ]:
function subdivided_edges(edges, offset)
    set = Vector()
    for e in 1:size(edges, 2)
        edge = edges[:, e]
        push!(set, [edge[1], e + offset])
        push!(set, [e + offset, edge[2]])
    end
    return hcat(set...)
end

In [ ]:
# triangulate with added center points and edge subdivision, keeping subdivided edges
points5 = vcat(points, centers_array, edge_centers)
points_map5 = collect(1:size(points5, 1))

offset = size(points, 1) + size(centers_array, 1)
sub_edges = subdivided_edges(edges, offset)

triangles5 = constrained_triangulation(points5, points_map5, sub_edges'[:,:])
edges5 = unique_edges(triangles5)

make_scene()
linesegments!(points4[vec(edges5), :], color=:orange)
linesegments!(points[vec(edges), :])
scatter!(edge_centers, markersize=4, color=:green)
scatter!(centers_array, markersize=4, color=:cyan)
scatter!(x, y, markersize=5, color=:red)